In [7]:
import os
import sys
import time
import numpy as np
from operator import itemgetter
from itertools import chain, islice
from functools import partial
from joblib import delayed, Parallel
import random
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')

root = os.path.dirname(os.path.abspath(""))

fortran_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "MC_singleSeg_singleTS")
sys.path.append(fortran_source_dir)
sys.path.append(os.path.join(root, "src", "python_framework_v02"))
sys.path.append(os.path.join(root, "src", "python_routing_v02"))

## network/reach utilities & routing module
import nhd_network_utilities_v02 as nnu
import nhd_io
import nhd_network
import mc_reach

# River network wave routing with t-route
This notebook illustrates how the t-route code base routes flood waves through large river networks. We created an experiment where a  pulse of lateral inflows is uniformly applied to and routed through the CONUS mainstem supernetwork. Experiment results are visulalized for Mississippi River Basin. 

In [9]:
def step_qlats(data, nsteps, qlat):
    
    q1 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q2 = np.full((len(data.index), nsteps//10), qlat, dtype='float32')
    q3 = np.full((len(data.index), nsteps//10), qlat, dtype='float32')
    q4 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q5 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q6 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q7 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q8 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q9 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    q10 = np.full((len(data.index), nsteps//10), 0, dtype='float32')
    
    q= np.concatenate((q1,q2,q3,q4,q5,q6,q7,q8,q9,q10),axis=1)
    
    ql = pd.DataFrame(q, index=data.index, columns=range(nsteps))
    
    return ql

In [ ]:
# specify the number of timesteps and the size of each timestep
nts = 8000
dt = 1800

# ------- STEP 1 ---------

print("Identifying supernetwork connections set")

test_folder = os.path.join(root, r"test")
geo_input_folder = os.path.join(test_folder, r"input", r"geo")
supernetwork = 'Mainstems_CONUS'

network_data = nnu.set_supernetwork_data(
    supernetwork=supernetwork, geo_input_folder=geo_input_folder
)

# select only the necessary columns of geospatial data, set the DataFrame index
cols = [v for c, v in network_data.items() if c.endswith("_col")]
data = nhd_io.read(network_data["geo_file_path"])
data = data[cols]
data = data.set_index(network_data["key_col"])

# mask NHDNetwork to isolate test network of choice
if "mask_file_path" in network_data:
    data_mask = nhd_io.read_mask(
        network_data["mask_file_path"],
        layer_string=network_data["mask_layer_string"],
    )
    data = data.filter(data_mask.iloc[:, network_data["mask_key"]], axis=0)

# sort index
data = data.sort_index()

# replace downstreams
data = nhd_io.replace_downstreams(data, network_data['downstream_col'], 0)

# generate a lateral inflow time series
qlats = step_qlats(data, nts, 10.0)

# extract downstream connections for each node
connections = nhd_network.extract_connections(data, network_data["downstream_col"])

print("supernetwork connections set created.")

# ------- STEP 2 ---------

print("Organizing segments into reaches.")

# reverse the network - track upstream connections
rconn = nhd_network.reverse_network(connections)

# isolate independent subnetworks
subnets = nhd_network.reachable_network(rconn)

# identify the segments in each subnetwork
subreachable = nhd_network.reachable(rconn)

# break each subnetwork into reaches 
subreaches = {}
for tw, net in subnets.items():
    path_func = partial(nhd_network.split_at_junction, net)
    subreaches[tw] = nhd_network.dfs_decomposition(net, path_func)  

print("Reach creation complete.")    

#  ------- STEP 3 ---------  

print("Running network routing computations")

# create a new DataFrame column containing the simulation timestep. 
data['dt'] = dt

# re-name DataFrame columns with specific headers. Needed b/c we reference the DataFrame by these column names exactly.
# 0: bw, 1: tw, 2: twcc, 3: dx, 4: n_manning 5: n_manning_cc, 6: cs, 7: s0, 8: qlat
column_rename = {
    network_data['length_col']: 'dx',
    network_data['topwidth_col']: 'tw',
    network_data['topwidthcc_col']: 'twcc',
    network_data['bottomwidth_col']: 'bw',
    network_data['manningncc_col']: 'ncc',
    network_data['slope_col']: 's0',
    network_data['ChSlp_col']: 'cs',
    network_data['manningn_col']: 'n'
}
data = data.rename(columns=column_rename)

# force variable type as float32. Needed b/c of FORTRAN interaction
data = data.astype('float32')

# loop through each subnetwork and execute the MC routing model
results = []
for twi, (tw, reach) in enumerate(subreaches.items(), 1):

    print(
                f"Now working on: {tw} ...",
                end = "", 
            )

    # get a list of segments in the subnetwork
    r = list(chain.from_iterable(reach))

    # prep parameter and lateral inflow data to be fed to routing model
    data_sub = data.loc[r, ['dt', 'bw', 'tw', 'twcc', 'dx', 'n', 'ncc', 'cs', 's0']].sort_index()
    qlat_sub = qlats.loc[r].sort_index()

    # compute the network routing, append results (flow, depth, and velocity)
    results.append(mc_reach.compute_network(
        nts, reach, subnets[tw], data_sub.index.values, data_sub.columns.values, data_sub.values, qlat_sub.values
        )
    )

# create a multi-index DataFrame with flow, depth, and velocity simulations
fdv_columns = pd.MultiIndex.from_product([range(nts), ['q', 'v', 'd']])
flowveldepth = pd.concat([pd.DataFrame(d, index=i, columns=fdv_columns) for i, d in results], copy=False)
flowveldepth = flowveldepth.sort_index()

print("Netowrk routing computations complete")

Identifying supernetwork connections set
supernetwork connections set created.
Organizing segments into reaches.
Reach creation complete.
Running network routing computations
Now working on: 20331522 ...Now working on: 209929 ...Now working on: 10850322 ...Now working on: 21412883 ...Now working on: 23864346 ...Now working on: 2297884 ...Now working on: 8272933 ...Now working on: 18257965 ...Now working on: 3712048 ...Now working on: 18524217 ...Now working on: 12169285 ...Now working on: 3766342 ...Now working on: 23940171 ...Now working on: 17562700 ...Now working on: 12135518 ...Now working on: 24903800 ...Now working on: 6170754 ...Now working on: 20105349 ...Now working on: 20849803 ...Now working on: 21906573 ...Now working on: 15634575 ...Now working on: 9619605 ...Now working on: 3468447 ...Now working on: 3172512 ...Now working on: 23875749 ...Now working on: 11947179 ...Now working on: 10367150 ...Now working on: 16875696 ...Now working on: 15588530 ...Now working on: 1320363

In [12]:
# grab the flow output data for a particular subnetwork
tailwater_id = 22811611   
network_segs = subreachable[tailwater_id]

# grab simulated flows for all segments in the network
# [i, j, k]:
# i = index values for each segment
# j = all times
# k = the parameter of interest
network_flows = flowveldepth.loc[flowveldepth.index.isin(network_segs), (slice(None), 'q')]

In [13]:
# create a GeoDataFrame of points for each link in the stream network
dat_geo = nhd_io.read(network_data["geo_file_path"])
dat_geo = dat_geo.set_index(network_data["key_col"])

if "mask_file_path" in network_data:
        data_mask = nhd_io.read_mask(
            network_data["mask_file_path"],
            layer_string=network_data["mask_layer_string"],
        )
        dat_geo = dat_geo.filter(data_mask.iloc[:, network_data["mask_key"]], axis=0)

# create a GeoData Frame        
gdf = geopandas.GeoDataFrame(dat_geo, geometry=geopandas.points_from_xy(dat_geo.lon, dat_geo.lat))

# clip the GeoDataFrame to include ONLY segments in the specified river network
network_gdf = gdf.loc[gdf.index.isin(network_segs)]


In [15]:
import gif
from mpl_toolkits.axes_grid1 import make_axes_locatable

# network geodata framee
network_gdf = gdf.loc[gdf.index.isin(network_segs)]

@gif.frame
def plot(tstep):
    
    first_flow = network_flows.loc[:, ((tstep), 'q')]
 
    joined_gdf = network_gdf.join(first_flow)
    joined_gdf = joined_gdf.rename(columns = {joined_gdf.columns[-1] : 'flow'})
    
    # create a figure showing spatial variations in simulated flow and the lateral flow loading
    fig = plt.figure(constrained_layout=True)
    gs = fig.add_gridspec(4, 3)
    ax1 = fig.add_subplot(gs[0:2, :]) # we will make a map on ax1
    ax2 = fig.add_subplot(gs[2:3,:])  # we will make a line plot on ax2

    # need to adjust the size and position of the map colorbar
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    # plot the spatial variation in simulated flow rate, across all stream segments in the network
    joined_gdf.plot(markersize = 5, 
                    column = "flow", 
                    cmap = 'PuBu', 
                    legend = True, 
                    vmin = 0, 
                    vmax = 20000,
                    ax = ax1, 
                    cax = cax, 
                    legend_kwds={'label': "Flow (cms)",
                             'orientation': "vertical"})

    # map title and axis controls
    ax1.set_title("Mississippi River Basin", size = 20)
    ax1.axis('off')

    # plot the timeseris of lateral inflow loading
    qlats.iloc[1][0:tstep].plot(ax = ax2, color = 'r', linewidth = 2) # ! make the 5 = tstep
    ax2.set_xlim([0, 8000])
    ax2.set_ylim([0, 11])
    ax2.set_title("Lateral Inflow Loading", size = 20)
    ax2.set_ylabel("Lateral Inflow (cms per node)", size = 14)
    ax2.set_xlabel("Simulated Timestep", size = 14)

    # adjsut the figure size
    fig.set_size_inches(10, 10)

frames = []
for i in range(0,8000,100):
    
    frame = plot(i)
    
    frames.append(frame)

gif.save(frames, "../doc/mississippi.gif", duration = 10)